In [18]:
import pandas as pd
import numpy as np
import sys, os , time, pickle
from tabulate import tabulate
from IPython.display import display

def find_match(string,list_des):
    return [value for value in list_des if string in value]

def plot_files(file_name):
    print("\nFile to show: ",file_name)
    table = pd.read_csv(file_name)
    display(table)
    #print(tabulate(table, headers='keys', tablefmt='rst')) #latex
    
def calculate_aux(archivo,what_to_do="mv",model=""):
    try:
        f = open(archivo)
        result_list = []
        for line in f.readlines():
            if what_to_do == "mv":
                if "ACC MV" in line: 
                    result_list.append(float(line.split(" ")[-1].strip()))
            elif what_to_do == 'epochs':
                if "Epochs to converge" in line and model in line: 
                    result_list.append(float(line.split(" ")[-1].strip()))
        f.close()
        return result_list
    except:
        return "-"
    
def compare_table(tipo,repre="normal", std=False):
    t = pd.DataFrame()
    t["Methods"] = ["Ann(tr)"]+methods_list
    t["Train"] = ["-"] + ["-" for _ in range(len(methods_list))]
    t["Test"] = ["-"] + ["-" for _ in range(len(methods_list))]
    if tipo == 'acc':
        try:
            t.loc[0,"Train"] = np.mean(calculate_aux(folder+"results.txt",what_to_do="mv"))
        except:
            try:
                t.loc[0,"Train"] = np.mean(calculate_aux(folder+"results_ind.txt",what_to_do="mv"))
            except:
                print("No se encontró acc")
            
    for k, method in enumerate(methods_list):
        aux = find_match(method,files)
        if tipo == "acc": #como normal?
            if std:
                test_aux = find_match("_train_std.csv",aux)
                t.loc[k+1,"Train"] = round(pd.read_csv(folder+test_aux[-1])["Accuracy"][0], 4)
                train_aux = find_match("_test_std.csv",aux)
                t.loc[k+1,"Test"] = round(pd.read_csv(folder+train_aux[-1])["Accuracy"][0], 4)
            else:
                test_aux = find_match("_train.csv",aux)
                t.loc[k+1,"Train"] = round(pd.read_csv(folder+test_aux[-1])["Accuracy"][0], 4)
                train_aux = find_match("_test.csv",aux)
                t.loc[k+1,"Test"] = round(pd.read_csv(folder+train_aux[-1])["Accuracy"][0], 4)

        elif "norm" in tipo.lower() or "js" in tipo.lower():
            if std:
                kl_aux = find_match("_train_std.csv",aux)
            else:
                kl_aux = find_match("_train.csv",aux)
            
            if len(kl_aux) != 0:                
                if repre=="R":
                    to_extract = "("+repre+") "+tipo+ " mean"
                elif repre=='G':
                    to_extract = "("+repre+") "+tipo
                try:
                    t.loc[k+1,"Train"] = round(pd.read_csv(folder+kl_aux[-1])[to_extract][0] ,4)
                except:
                    pass
        elif tipo == "epochs":
            methods_list_aux = ["soft-MV","hard-MV", "D&S", "Raykar", "Ours Global","Ours Individual","2", "3"]
            t.loc[k+1,"Train"]=np.mean(calculate_aux(folder+"results.txt",what_to_do="epochs",model=methods_list_aux[k]))
    return t

def get_value(lista, name, std=False):
    i = 0
    if std:
        i = 1
    return np.asarray([value[i][name][0] if type(value[i])== pd.DataFrame else -9999 for value in lista] )

def create_df(method_l,T_try, value, file_re = "./results.txt"):
    t = pd.DataFrame(np.zeros((len(method_l)+1,len(T_try))))    
    t.columns = T_try
    t.index = ["Ann(tr)"]+method_l
    t_std = t.copy()
    
    if value.lower() == "accuracy" or "F1" in value.lower():
        if len(calculate_aux(file_re,what_to_do="mv")) > len(T_try) +1:
            aux2 = calculate_aux(file_re,what_to_do="mv")
            aux = [np.mean(aux2[i:30*(i+1)]) for i in range(0,len(aux2),30)]
            aux2 = [np.std(aux2[i:30*(i+1)]) for i in range(0,len(aux2),30)]
        else:
            aux = calculate_aux(file_re,what_to_do="mv")
            aux2 = np.zeros((len(T_try)))
        t.loc["Ann(tr)",:] = aux
        t_std.loc["Ann(tr)",:] = aux2
        for method_l in methods_list:
            t.loc[method_l,:] = get_value(dic_res[method_l]["test"], value)
            t_std.loc[method_l,:] = get_value(dic_res[method_l]["test"], value, std=True)

    elif "inertia" in value.lower() or "iner" in value.lower():
        for method_l in methods_list:
            if "Ours" in method_l:
                t.loc[method_l,:] = get_value(dic_res[method_l]["testAux"], value)
                
    else:
        t.index = ["D_t"]+method_l
        t.loc["D_t",:] = D_t
        for method_l in methods_list:
            t.loc[method_l,:] = get_value(dic_res[method_l]["train"], value)
            t_std.loc[method_l,:] = get_value(dic_res[method_l]["train"], value, std=True)
    return t.round(4), t_std.round(4)

!ls

LabelMe  LabelMe_v2  Plot_Results.ipynb  simple_T  simple_T_re


In [12]:
!ls LabelMe
folder = "LabelMe/"

files = sorted(os.listdir(folder))
methods_list = list(set([f.split("_")[1] for f in os.listdir(folder) if (len(f.split("_")) >2) and ("UpperBound" not in f)]))
methods_list

LabelMe_DS_test.csv
LabelMe_DS_test_std.csv
LabelMe_DS_train.csv
LabelMe_DS_train_std.csv
LabelMe_hardMV_test.csv
LabelMe_hardMV_test_std.csv
LabelMe_hardMV_train.csv
LabelMe_hardMV_train_std.csv
LabelMe_OursGlobal_testAux.csv
LabelMe_OursGlobal_testAux_std.csv
LabelMe_OursGlobal_test.csv
LabelMe_OursGlobal_test_std.csv
LabelMe_OursGlobal_trainAnn.csv
LabelMe_OursGlobal_trainAnn_std.csv
LabelMe_OursGlobal_train.csv
LabelMe_OursGlobal_train_std.csv
LabelMe_OursIndividualK_testAux.csv
LabelMe_OursIndividualK_testAux_std.csv
LabelMe_OursIndividualK_test.csv
LabelMe_OursIndividualK_test_std.csv
LabelMe_OursIndividualK_trainAnn.csv
LabelMe_OursIndividualK_trainAnn_std.csv
LabelMe_OursIndividualK_train.csv
LabelMe_OursIndividualK_train_std.csv
LabelMe_OursIndividualT_testAux.csv
LabelMe_OursIndividualT_testAux_std.csv
LabelMe_OursIndividualT_test.csv
LabelMe_OursIndividualT_test_std.csv
LabelMe_OursIndividualT_trainAnn.csv
LabelMe_OursIndividualT_trainAnn_std.csv
LabelMe_OursIndividualT_trai

['OursGlobal',
 'OursIndividualK',
 'Raykar',
 'softMV',
 'OursIndividualT',
 'DS',
 'hardMV']

In [20]:
t_acc = compare_table("acc")
t_acc_std = compare_table("acc", std=True)

t_norm_R = compare_table("NormF","R")
t_norm_R_std = compare_table("NormF","R", std=True)
t_js_R = compare_table("JS","R")
t_js_R_std = compare_table("JS","R",  std=True)

t_norm_G = compare_table("NormF","G")
t_norm_G_std = compare_table("NormF","G", std=True)
t_js_G = compare_table("JS","G")
t_js_G_std = compare_table("JS","G", std=True)

t_epochs = compare_table("epochs")

In [14]:
t_acc.merge(t_acc_std.iloc[:,1:], left_index=True, right_index=True)

,Methods,Train_x,Test_x,Train_y,Test_y
0,Ann(tr),0.769,-,0.769,-
1,OursGlobal,0.8464,0.8269,0.0145,0.0137
2,OursIndividualK,0.8067,0.8332,0.0063,0.0051
3,Raykar,0.8547,0.8304,0.0093,0.011
4,softMV,0.8322,0.8217,0.0085,0.0108
5,OursIndividualT,0.8385,0.8378,0.0063,0.0089
6,DS,0.8443,0.8353,0.0072,0.009
7,hardMV,0.8062,0.802,0.0085,0.0112


In [15]:
from IPython.display import display
display(t_js_R.merge(t_js_R_std.iloc[:,1:], left_index=True, right_index=True))
display(t_norm_R.merge(t_norm_R_std.iloc[:,1:], left_index=True, right_index=True))

,Methods,Train_x,Test_x,Train_y,Test_y
0,Ann(tr),-,-,-,-
1,OursGlobal,0.2273,-,0.0031,-
2,OursIndividualK,0.2372,-,0.0075,-
3,Raykar,0.2949,-,0.0013,-
4,softMV,0.2163,-,0,-
5,OursIndividualT,0.2354,-,0.0033,-
6,DS,0.153,-,0,-
7,hardMV,0.2252,-,0,-


,Methods,Train_x,Test_x,Train_y,Test_y
0,Ann(tr),-,-,-,-
1,OursGlobal,0.1818,-,0.0022,-
2,OursIndividualK,0.1949,-,0.0058,-
3,Raykar,0.1761,-,0.0006,-
4,softMV,0.172,-,0,-
5,OursIndividualT,0.1859,-,0.0023,-
6,DS,0.1394,-,0,-
7,hardMV,0.1812,-,0,-


In [16]:
display(t_js_G.merge(t_js_G_std.iloc[:,1:], left_index=True, right_index=True))
display(t_norm_G.merge(t_norm_G_std.iloc[:,1:], left_index=True, right_index=True))

,Methods,Train_x,Test_x,Train_y,Test_y
0,Ann(tr),-,-,-,-
1,OursGlobal,0.0452,-,0.0042,-
2,OursIndividualK,0.0919,-,0.0078,-
3,Raykar,0.2591,-,0.0006,-
4,softMV,0.0242,-,0,-
5,OursIndividualT,0.0621,-,0.0042,-
6,DS,0.0357,-,0,-
7,hardMV,0.0347,-,0,-


,Methods,Train_x,Test_x,Train_y,Test_y
0,Ann(tr),-,-,-,-
1,OursGlobal,0.076,-,0.0038,-
2,OursIndividualK,0.1032,-,0.0062,-
3,Raykar,0.1681,-,0.0003,-
4,softMV,0.0566,-,0,-
5,OursIndividualT,0.0851,-,0.0035,-
6,DS,0.0468,-,0,-
7,hardMV,0.0736,-,0,-


In [21]:
t_epochs

,Methods,Train,Test
0,Ann(tr),-,-
1,OursGlobal,9.03333,-
2,OursIndividualK,11,-
3,Raykar,9.9,-
4,softMV,3.93333,-
5,OursIndividualT,6.77667,-
6,DS,12.6742,-
7,hardMV,15.4923,-


## T change

In [5]:
folder = "./simple_T_re//"

In [6]:
to_check = [100,500,1500,3500,6000,10000]

if 'simple' in folder.lower():
    T_data = 5 #for now is this..
    N = 2500
elif "cifar" in folder.lower():
    T_data = 3
    N = 50000
D_t = np.asarray([N*T_data/T for T in to_check])

files_f = [f for f in os.listdir(folder) if ".pickle" in f]
dic_res = {}
for file in files_f:
    #if len(file.split("_")) >2:
    _, method_name, method_type = (file.split(".pickle")[0]).split("_")
    with open(folder+file, 'rb') as handle:
        if method_name in dic_res:
            dic_res[method_name][method_type] = pickle.load(handle)
        else:
            dic_res[method_name] = {}
            dic_res[method_name][method_type] = pickle.load(handle)
methods_list = list(dic_res.keys())
methods_list

['hardMV',
 'OursGlobal',
 'OursIndividualK',
 'Raykar',
 'OursIndividualT',
 'DS',
 'softMV']

In [7]:
t, t_std = create_df(methods_list, to_check,"Accuracy", file_re=folder+'results.txt')
t.merge(t_std.iloc[:,1:], left_index=True, right_index=True)

,100,500_x,1500_x,3500_x,6000_x,10000_x,500_y,1500_y,3500_y,6000_y,10000_y
Ann(tr),0.7211,0.7201,0.7197,0.7196,0.7210,0.7229,0.0103,0.0100,0.0098,0.0093,0.0075
hardMV,0.7939,0.8046,0.7795,0.7692,0.7825,0.8065,0.0515,0.0689,0.0696,0.0699,0.0480
OursGlobal,0.9133,0.9093,0.8941,0.8891,0.9071,0.9122,0.0193,0.1022,0.1010,0.0158,0.0168
OursIndividualK,0.9418,0.9459,0.9430,0.9172,0.8775,0.7960,0.0041,0.0080,0.0356,0.0719,0.1216
Raykar,0.9405,0.9349,0.9258,0.8435,0.7597,0.7156,0.0076,0.0123,0.1267,0.1223,0.1044
OursIndividualT,0.9416,0.9405,0.9171,0.9256,0.9172,0.8479,0.0073,0.0137,0.0271,0.0436,0.0879
DS,0.9403,0.9350,0.9289,0.9184,0.8923,0.8598,0.0100,0.0130,0.0202,0.0386,0.0690
softMV,0.6825,0.6823,0.6786,0.6619,0.6657,0.6771,0.0344,0.0518,0.0244,0.0307,0.0299


In [8]:
t, t_std = create_df(methods_list,to_check, "(R) JS mean")
#t, t_std = create_df(methods_list,to_check, "(R) NormF mean")
t.merge(t_std.iloc[:,1:], left_index=True, right_index=True)

,100,500_x,1500_x,3500_x,6000_x,10000_x,500_y,1500_y,3500_y,6000_y,10000_y
hardMV,0.0868,0.0856,0.0869,0.0876,0.0870,0.0868,0.0031,0.0025,0.0015,0.0013,0.0012
OursGlobal,0.1059,0.1086,0.1144,0.1181,0.1117,0.1114,0.0170,0.0368,0.0297,0.0156,0.0159
OursIndividualK,0.0196,0.0208,0.0473,0.0924,0.1389,0.1921,0.0043,0.0053,0.0053,0.0098,0.0159
Raykar,0.0144,0.0558,0.1137,0.1569,0.1727,0.1820,0.0011,0.0015,0.0050,0.0035,0.0031
OursIndividualT,0.0175,0.0221,0.0541,0.1127,0.1507,0.1927,0.0028,0.0065,0.0087,0.0077,0.0191
DS,0.0120,0.0671,0.1771,0.2515,0.2796,0.3049,0.0024,0.0027,0.0023,0.0033,0.0022
softMV,0.0871,0.0863,0.0869,0.0875,0.0871,0.0870,0.0027,0.0017,0.0010,0.0008,0.0010


In [7]:
t, t_std = create_df(methods_list,to_check,"(G) JS")
#t, t_std = create_df(methods_list,to_check,"(G) NormF")
t.merge(t_std.iloc[:,1:], left_index=True, right_index=True)

,100,500_x,1500_x,3500_x,6000_x,10000_x,500_y,1500_y,3500_y,6000_y,10000_y
softMV,0.0139,0.0140,0.0141,0.0137,0.0151,0.0141,0.0000,0.0000,0.0000,0.0000,0.0000
OursGlobal,0.0510,0.0475,0.0611,0.0618,0.0653,0.0571,0.0138,0.0349,0.0146,0.0246,0.0102
DS,0.0001,0.0025,0.0053,0.0227,0.0401,0.0483,0.0000,0.0000,0.0000,0.0000,0.0000
OursIndividualT,0.0036,0.0017,0.0103,0.0046,0.0089,0.0263,0.0006,0.0043,0.0006,0.0004,0.0096
hardMV,0.0155,0.0182,0.0143,0.0140,0.0167,0.0176,0.0000,0.0000,0.0000,0.0000,0.0000
Raykar,0.0021,0.0218,0.0587,0.0913,0.1076,0.1090,0.0003,0.0003,0.0007,0.0024,0.0021
OursIndividualK,0.0061,0.0025,0.0024,0.0086,0.0132,0.0301,0.0014,0.0016,0.0023,0.0012,0.0100


In [ ]:
print(tabulate(t,headers='keys',tablefmt='latex'))

In [8]:
t, _ = create_df(methods_list,to_check,"Iner JS")
#t, _ = create_df(methods_list,to_check,"Iner NormF")
t.merge(t_std.iloc[:,1:], left_index=True, right_index=True)

,100,500_x,1500_x,3500_x,6000_x,10000_x,500_y,1500_y,3500_y,6000_y,10000_y
Ann(tr),0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
softMV,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
OursGlobal,0.2539,0.2087,0.1349,0.2122,0.2082,0.2062,0.0138,0.0349,0.0146,0.0246,0.0102
DS,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
OursIndividualT,0.3096,0.3609,0.3964,0.6011,0.6713,0.7111,0.0006,0.0043,0.0006,0.0004,0.0096
hardMV,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
Raykar,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0003,0.0003,0.0007,0.0024,0.0021
OursIndividualK,0.2976,0.3421,0.3874,0.5060,0.5801,0.6641,0.0014,0.0016,0.0023,0.0012,0.0100
